In [57]:
!pip install scikit-learn joblib imblearn xgboost

In [58]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

In [59]:
# Handling Missing Values
df_heart = pd.read_csv('/content/heart_dataset.csv')
display(df_heart.head())
display(df_heart.isnull().sum())


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [60]:
target_column = 'target'

In [61]:
df_cleaned = df_heart.copy()

# Replace '?' with NaN
df_cleaned.replace("?", np.nan, inplace=True)

# Convert numeric values
for col in df_cleaned.columns:
    if col != target_column:
        df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors="ignore")

display(df_cleaned.head())
display(df_cleaned.info())


/tmp/ipython-input-1508359260.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors="ignore")


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


None

In [62]:
# Encode categorical features
object_cols = df_cleaned.select_dtypes(include="object").columns
if not object_cols.empty:
    print(f"Found object columns to encode: {list(object_cols)}")
    for col in object_cols:
        if col != target_column:
            le = LabelEncoder()
            df_cleaned[col] = le.fit_transform(df_cleaned[col].astype(str))
    print("DataFrame head after Label Encoding:")
    display(df_cleaned.head())
else:
    print("No object columns found for Label Encoding.")
    print("All columns are already numeric.")

display(df_cleaned.info())

No object columns found for Label Encoding.
All columns are already numeric.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


None

In [63]:
X = df_cleaned.drop(columns=[target_column])
y = df_cleaned[target_column]

print("Features (X) head:")
display(X.head())
print("Target (y) head:")
display(y.head())

Features (X) head:


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


Target (y) head:


,target
0,1
1,1
2,1
3,1
4,1


In [64]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

print("Data split into training and testing, and training features scaled.")

Data split into training and testing, and training features scaled.


In [65]:
# Applying SMOTE for class balancing

print("Class distribution BEFORE SMOTE:")
display(y_train.value_counts())

sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_s, y_train)

print("SMOTE applied to the training data.")

print("Class distribution AFTER SMOTE:")
display(y_train_resampled.value_counts())

Class distribution BEFORE SMOTE:


,count
target,
1,132
0,110


SMOTE applied to the training data.
Class distribution AFTER SMOTE:


,count
target,
1,132
0,132


In [68]:
# Define the disease key and name for saving metadata
disease_key = 'heart'
disease_name = 'Heart Disease'

models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000)
}

best_model = None
best_f1 = -1
best_name = ""

# --- TRAIN MODELS + SELECT BEST ---
print("\n--- Training and evaluating candidate models ---")
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_resampled, y_train_resampled)
    pred = model.predict(X_test_s)
    f1 = f1_score(y_test, pred, average='weighted')
    accuracy = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred, average='weighted')
    precision = precision_score(y_test, pred, average='weighted')

    print(f"✔ {name} → F1 = {f1:.4f}, Accuracy = {accuracy:.4f}, Recall = {recall:.4f}, Precision = {precision:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_name = name
        best_model = model

print(f"\n Best Model Selected: {best_name} with F1-score: {best_f1:.4f}")

print(f"\n Best Model Selected for Heart Disease: {best_name} with F1-score: {best_f1:.4f}")




--- Training and evaluating candidate models ---
Training Random Forest...
✔ Random Forest → F1 = 0.8172, Accuracy = 0.8197, Recall = 0.8197, Precision = 0.8262
Training SVM...
✔ SVM → F1 = 0.8033, Accuracy = 0.8033, Recall = 0.8033, Precision = 0.8033
Training XGBoost...
✔ XGBoost → F1 = 0.7351, Accuracy = 0.7377, Recall = 0.7377, Precision = 0.7390
Training Logistic Regression...
✔ Logistic Regression → F1 = 0.7839, Accuracy = 0.7869, Recall = 0.7869, Precision = 0.7920

 Best Model Selected: Random Forest with F1-score: 0.8172

 Best Model Selected for Heart Disease: Random Forest with F1-score: 0.8172


In [69]:
print("\n Calibrating probability output for Heart Disease...")

# Use y_train_resampled for min_class calculation
min_class = pd.Series(y_train_resampled).value_counts().min()
cv_folds = max(2, min(5, min_class))

calibrated_model = CalibratedClassifierCV(best_model, cv=cv_folds, method="sigmoid")
calibrated_model.fit(X_train_resampled, y_train_resampled)

print("✔ Calibration complete.")


print("\n--- Saving Heart Disease Model + Metadata ---")
out_dir = "/content/model_data"

# Save calibrated model and scale
joblib.dump(calibrated_model, f"{out_dir}/{disease_key}_model_calibrated.pkl")
joblib.dump(scaler, f"{out_dir}/{disease_key}_scaler.pkl")

# Metadata
metadata = {
    "features": X.columns.tolist(),
    "target": target_column,
    "disease": disease_name,
    "best_model": best_name,
    "feature_medians": X.median().to_dict()
}

joblib.dump(metadata, f"{out_dir}/{disease_key}_metadata.pkl")

print(f"💾 Saved calibrated model: {disease_key}_model_calibrated.pkl")
print(f"💾 Saved scaler: {disease_key}_scaler.pkl")
print(f"💾 Saved metadata: {disease_key}_metadata.pkl")

print("\n🎉 Heart Disease Model Trained + Calibrated Successfully!")



 Calibrating probability output for Heart Disease...
✔ Calibration complete.

--- Saving Heart Disease Model + Metadata ---
💾 Saved calibrated model: heart_model_calibrated.pkl
💾 Saved scaler: heart_scaler.pkl
💾 Saved metadata: heart_metadata.pkl

🎉 Heart Disease Model Trained + Calibrated Successfully!
